<a href="https://colab.research.google.com/github/arnabd64/Langchain-Guides/blob/main/notebooks/Langchain_Day_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --no-cache-dir --progress-bar off \
    langchain==0.2.10 \
    langchain_community==0.2.10 \
    python-dotenv \
    > install.log

In [2]:
import os
import dotenv

if not dotenv.load_dotenv("./.env"):
    raise FileNotFoundError

In [3]:
from langchain_core.runnables import RunnableLambda, RunnableSequence, RunnableBranch, RunnableParallel, RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.llms.ollama import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain.globals import set_debug

set_debug(True)

In [4]:
# Area of Circle Runnable
# Area of Circle = pi * r * r
import math

circle_area = RunnableSequence(
    first=RunnableLambda(lambda r: r * r, name="Radius-Squared"),
    last=RunnableLambda(lambda y: y * math.pi, name="Multiply Pi"),
    name="Area of Circle"
)
output = circle_area.invoke(7)
print("Area of the Circle = ", output)

[chain/start] [chain:Area of Circle] Entering Chain run with input:
{
  "input": 7
}
[chain/start] [chain:Area of Circle > chain:Radius-Squared] Entering Chain run with input:
{
  "input": 7
}
[chain/end] [chain:Area of Circle > chain:Radius-Squared] s] Exiting Chain run with output:
{
  "output": 49
}
[chain/start] [chain:Area of Circle > chain:Multiply Pi] Entering Chain run with input:
{
  "input": 49
}
[chain/end] [chain:Area of Circle > chain:Multiply Pi] s] Exiting Chain run with output:
{
  "output": 153.93804002589985
}
[chain/end] [chain:Area of Circle] s] Exiting Chain run with output:
{
  "output": 153.93804002589985
}
Area of the Circle =  153.93804002589985


In [5]:
# pipe operator
circle_area = (
    RunnableLambda(lambda r: r * r)
    | RunnableLambda(lambda y: y * math.pi)
)
output = circle_area.invoke(5.612)
print("Area of Circle = ", output)

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "input": 5.612
}
[chain/start] [chain:RunnableSequence > chain:RunnableLambda] Entering Chain run with input:
{
  "input": 5.612
}
[chain/end] [chain:RunnableSequence > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": 31.494544
}
[chain/start] [chain:RunnableSequence > chain:RunnableLambda] Entering Chain run with input:
{
  "input": 31.494544
}
[chain/end] [chain:RunnableSequence > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": 98.9430280585605
}
[chain/end] [chain:RunnableSequence] s] Exiting Chain run with output:
{
  "output": 98.9430280585605
}
Area of Circle =  98.9430280585605


In [7]:
# description prompt
description_prompt = {
    "template": "You are an assistant tasked with gathering information about a product that an user wants to know about. Get the information for: {product}",
    "input_variables": ["product"]
}

# LLM config
config = {
    "name": "Ollama-Local",
    "base_url": os.getenv("HOST"),
    "model": os.getenv("MODEL"),
    "temperature": float(os.getenv("TEMPERATURE")),
    "timeout": int(os.getenv("TIMEOUT")),
    "keep_alive": 3600
}

# build a chain to gather information
query_chain = (
    PromptTemplate(**description_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

In [8]:
# Pros Chain
#Prompt template
pros_prompt = {
    "input_variables": ["description"],
    "template": "Write down exactly 5 points on why should the user purchase a product according to the following product description:\n{description}"
}

# build the chain
pros_chain = (
    PromptTemplate(**pros_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

In [9]:
# Cons Chain
# Prompt
cons_prompt = {
    "input_variables": ["description"],
    "template": "Write down exactly 5 points on why the user should not purchase a product according to the following product description:\n{description}"
}

# cons chain
cons_chain = (
    PromptTemplate(**cons_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

In [10]:
# Combine the Chains
combine_prompt = {
    "input_variables": ["pros", "cons"],
    "template": "You are tasked with summarizing the advantages and disadvantages of purchasing a product from the given list: Firstly, the advatages:\n{pros} and finally the disadvantages:\n{cons}"
}

# combine chain
combine_chain = (
    PromptTemplate(**combine_prompt)
    | Ollama(**config)
    | StrOutputParser()
)

In [11]:
final_chain = (
    query_chain
    | RunnableParallel(pros=pros_chain, cons=cons_chain)
    | combine_chain
)

In [12]:
response = final_chain.invoke({"product": "Porsche 911 Turbo GT"})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "product": "Porsche 911 Turbo GT"
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "product": "Porsche 911 Turbo GT"
}
[chain/end] [chain:RunnableSequence > prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:Ollama-Local] Entering LLM run with input:
{
  "prompts": [
    "You are an assistant tasked with gathering information about a product that an user wants to know about. Get the information for: Porsche 911 Turbo GT"
  ]
}
[llm/end] [chain:RunnableSequence > llm:Ollama-Local] [160.58s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " I have researched and compiled some key details about the Porsche 911 Turbo GT for you. Please note that specifications may vary depending on the year of production, so it's always a good idea to check with an authorized Porsche dealer or

In [13]:
print(response)

 Advantages of purchasing a Porsche 911 Turbo S or Turbo S Exclusive Series:
1. Unmatched Performance: With breathtaking speed and impressive top speed, the car offers an unparalleled driving experience.
2. Advanced Technology: The PDK automatic transmission and Rear-wheel steering system enhance performance and handling agility.
3. Superior Handling and Comfort: The PASM sports chassis provides excellent road grip and stability, while the interior ensures optimal comfort during high-speed drives.
4. Exclusive Design: The unique exterior design sets the car apart from other sports cars, and the Turbo S Exclusive Series offers even more exclusivity with carbon fiber components and Alcantara upholstery.
5. Premium Quality and Craftsmanship: Porsche's dedication to quality ensures a luxurious driving experience that exceeds expectations.

Disadvantages of purchasing a Porsche 911 Turbo S or Turbo S Exclusive Series:
1. Incorrect Product Name: The product described is not the Porsche 911 T

In [14]:
chain_config = {
    # Set the prompts
    "prompts": {
        # prompt to get a description of the product
        "description": {
            "name": "Prompt-Generate-Product_Description",
            "input_variables": ["product"],
            "template": "You are an assistant tasked with gathering information about a product that an user wants to know about. Get the information for: {product}"
        },

        # prompt to get the advantages of a purchase
        "pros": {
            "name": "Prompt-Generate-Advantages",
            "input_variables": ["description"],
            "template": "List exactly 5 reasons why the user should purchase the product described as:\n{description}"
        },

        # prompt to get the disadvantages of a purchse
        "cons": {
            "name": "Prompt-Generate-Disadvantages",
            "input_variables": ["description"],
            "template": "List exactly 5 reasons why the user should not purchase the product described as:\n{description}"
        },

        # prompt to combine both the pros and cons into a single summary
        "combine": {
            "name": "Prompt-Summarize-Outputs",
            "input_variables": ["pros", "cons"],
            "template": "Write down 3 points each on the Advantages and Disadvantages of making a purchase for a product. The advantages of a purcahse are:\n{pros} and the disadvantages of a purcashe are:\n{cons}"
        }
    },

    "llm-config": {
        "name": "Ollama-Local",
        "base_url": os.getenv("HOST"),
        "model": os.getenv("MODEL"),
        "temperature": float(os.getenv("TEMPERATURE")),
        "timeout": int(os.getenv("TIMEOUT")),
        "keep_alive": 3600
    }
}

In [15]:
chain_pipe = (
    # 1. Description Generator Chain
    PromptTemplate(**chain_config["prompts"]["description"])
    | Ollama(**chain_config["llm-config"])
    | StrOutputParser()

    # 2. Parallel chains
    | RunnableParallel(
        # 2A. Generate Advantages
        pros = (
            PromptTemplate(**chain_config["prompts"]["pros"])
            | Ollama(**chain_config["llm-config"])
            | StrOutputParser()
        ),
        # 2B. Generate Disadvantages
        cons = (
            PromptTemplate(**chain_config["prompts"]["cons"])
            | Ollama(**chain_config["llm-config"])
            | StrOutputParser()
        )
    )

    # 3. Combine the Advantages & Disadvantages
    | PromptTemplate(**chain_config["prompts"]["combine"])
    | Ollama(**chain_config["llm-config"])
    | StrOutputParser()
)

In [16]:
set_debug(False)
response = chain_pipe.invoke({"product": "Porsche 911 Carrera RS"})

In [17]:
print(response)

 Advantages of Making a Purchase:
1. Satisfaction and Enjoyment: Buying a product can bring satisfaction, as it meets a need or desire. Whether it's for practical use, entertainment, or self-expression, owning the product can provide enjoyment.
2. Ownership and Customization: With ownership comes the ability to customize the product according to personal preferences and needs. This can make the product more personal and tailored to the user's lifestyle.
3. Status and Prestige: Some products carry a certain status or prestige, which can enhance one's image or perceived value. This is particularly true for luxury items or popular brands.

Disadvantages of Making a Purchase:
1. Financial Commitment: Making a purchase involves spending money, which may not always be readily available. Additionally, some products require ongoing costs for maintenance, insurance, and other expenses.
2. Limited Usefulness: Not all purchases are equally useful or beneficial. Some products may quickly become ou

In [20]:
chain_subchain = RunnableSequence(
    # 1. Product Description Generator Sub-Chain
    RunnableSequence(
        PromptTemplate(**chain_config["prompts"]["description"]),
        Ollama(**chain_config["llm-config"]),
        StrOutputParser(),
        name="Chain-Generate-Product_Description"
    ),

    # 2. Generate Advantages & Disadvantages
    RunnableParallel(
        pros = RunnableSequence(
            # 2A. Generate the Advantages
            PromptTemplate(**chain_config["prompts"]["pros"]),
            Ollama(**chain_config["llm-config"]),
            StrOutputParser(),
            name="Chain-Generate-Advantages"
        ),
        cons = RunnableSequence(
            # 2B. Generate the Disadvantages
            PromptTemplate(**chain_config["prompts"]["cons"]),
            Ollama(**chain_config["llm-config"]),
            StrOutputParser(),
            name="Chain-Generate-Disadvantages"
        )
    ),

    # 3. Combine the Results
    RunnableSequence(
        PromptTemplate(**chain_config["prompts"]["combine"]),
        Ollama(**chain_config["llm-config"]),
        StrOutputParser(),
        name="Chain-Summarize-Output"
    ),
    name="Parent-Chain"
)

In [23]:
set_debug(True)
chain_subchain.invoke({"product": "Porsche Carrera RS"})

[chain/start] [chain:Parent-Chain] Entering Chain run with input:
{
  "product": "Porsche Carrera RS"
}
[chain/start] [chain:Parent-Chain > prompt:Prompt-Generate-Product_Description] Entering Prompt run with input:
{
  "product": "Porsche Carrera RS"
}
[chain/end] [chain:Parent-Chain > prompt:Prompt-Generate-Product_Description] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:Parent-Chain > llm:Ollama-Local] Entering LLM run with input:
{
  "prompts": [
    "You are an assistant tasked with gathering information about a product that an user wants to know about. Get the information for: Porsche Carrera RS"
  ]
}
[llm/end] [chain:Parent-Chain > llm:Ollama-Local] [135.79s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": " I have gathered some key information about the Porsche Carrera RS for you:\n\n1. **Model**: Porsche Carrera RS (Rennsport)\n\n2. **Years Produced**: 1973-1974 (first generation), 1989-1994 (second generation)\n\n3. **Body 

" To summarize the advantages of purchasing a Porsche Carrera RS:\n\n1. **Racing Heritage**: The car's racing background and limited production runs make it an attractive choice for enthusiasts who appreciate historical significance and exclusivity.\n2. **Superior Performance**: With powerful engines, quick acceleration, and responsive handling, the Carrera RS offers a thrilling driving experience.\n3. **Exclusivity and Collector Value**: Due to its limited production, owning a Porsche Carrera RS grants you membership in an exclusive group of enthusiasts and potential increases in value over time for collectors.\n4. **Iconic Status**: The Carrera RS is a timeless icon in the world of sports cars and offers the satisfaction of owning an automotive legend.\n\nThe disadvantages of purchasing a Porsche Carerra RS include:\n\n1. **High Cost**: The high cost of ownership, including insurance, maintenance, and fuel expenses, may make it difficult for some buyers to justify the expense.\n2. **